In [12]:
from input_output import *
from util import *
import altair as alt
alt.data_transformers.disable_max_rows()

print("Available datasets:\n\t" + ',\n\t'.join(os.listdir(sims_root())))
results_name = input("Input the name of a dataset: ")
# construct a PatchySimResultSet object from the specified directory
results = choose_results(results_name)

if len(results.targets) > 1:
    target_name = input(f"Input the name of an analysis target ({','.join(results.targets.keys())}): ")
else:
    target_name = list(results.targets.keys())[0]
target = results.targets[target_name]

if len(results.runs) > 1:
    eg = input(f"Enter the export group ({','.join(results.export_groups())}):") 
else:
    eg = results.runs[0].name

Available datasets:
	Allosteric_Pyramid_Full_Fall22,
	restart_bad_trajs.sh,
	Allosteric_Pyramid_Partial_Fall22,
	AlloSolidCube_X5_4_12Dec22,
	AlloSolidCube_X5_1_12Dec22,
	AlloSolidCube_X3_Singlet_12Dec22,
	check_traj_sizes.sh,
	NoAlloPyramid_14Oct22,
	AlloPyramid_X4_14Mar23,
	jobinfo.txt,
	AlloSolidCube_X1_12Dec22


Input the name of a dataset:  AlloPyramid_X4_14Mar23


In [13]:
# narrow type = set of parameters s if x.find("_duplicate") > -1 else xpecifying bonding angles between patches
nt = int(input(f"Input the narrow type of the data ({', '.join([f'{nt}' for nt in results.narrow_types])}): "))
# cutoff = the lower limit for counting clusters as partially assembled forms of the target
# cutoff = 1 -> only count fully assembled clusters
temp = float(input(f"Input temperature"))
cutoff = float(input("Input the graph match cutoff: "))
relative = input("Calculate relative yields?") == "True"
overreach = input("Include overreaching graphs in yield calculations?") == "True"

Input the narrow type of the data (0, 1, 2, 3):  0
Input temperature 0.01
Input the graph match cutoff:  1
Calculate relative yields? True
Include overreaching graphs in yield calculations? False
Enter resample  frequency (in kilosteps, should be >= to the observable print interval): 


In [14]:
df = results.get_flat_yields(target_name, cutoff, overreach,filters={'nt':nt, 'temp':temp})

In [15]:
ymax = df['num_assemblies'] * target['rel_count']
if relative:
    df['yield'] =  df['yield'] / ymax
    ymax = 1.0
    x_axis_format = '.0%'
else:
    x_axis_format = '.00'

In [16]:
graphtitle = f"Duplicates of Experiment ({results_name}"
if len(results.runs) > 1:
    graphtitle += f"Export Group: {eg},"
graphtitle += f"NT: {nt}, T{temp}) Yield"
if len(results.targets) > 1:
    graphtitle += f" (Target Topology: {target_name})"
alt.Chart(df, title=graphtitle).mark_line().encode(
    x=alt.X('time:Q', title="Time (megasteps)", axis=alt.Axis(tickCount=5)),
    y=alt.Y(
            "yield:Q",
            title="Yield",
            scale=alt.Scale(domain=(0,ymax)),
            axis=alt.Axis(format=x_axis_format)
        ),
    color=alt.Color('duplicate:O', scale=alt.Scale(scheme="accent"), title="Duplicate #")
)

/home/jrevan21/.conda/envs/polycubes2/lib/python3.10/site-packages/altair/utils/core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

,nt,temp,duplicate,tidx,yield,shape,num_assemblies,time
